# Highway Corridors 

Where are the opportunities for better transit service on highways? Seattle has a lot of highways with transit that are more express service style, which complement the local transit service. 

What highways would benefit from more complementary bus express service? If Caltrans were to launch its express bus line on the highways, where would you put it?

Draw quadrant? See which highway corridors have a lot of parallel routes per mile, but few competitive routes per mile? % parallel routes vs % competitive routes? Both are normalized metrics.

In [ ]:
import altair as alt
import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

import setup_corridors_stats
from create_parallel_corridors import IMG_PATH, DATA_PATH
from shared_utils import geography_utils, map_utils, styleguide
from shared_utils import calitp_color_palette as cp

alt.themes.register("calitp_theme", styleguide.calitp_theme)

catalog = intake.open_catalog("./*.yml")

In [ ]:
def process_hwy_stats(df):
    # parallel routes / competitive routes per mile
    # or percents?
    # right now, highway_length is in feet
    df = df.assign(
        parallel_per_mi = (df.num_parallel.divide(df.highway_length) * 
                           geography_utils.FEET_PER_MI),
        competitive_per_mi = (df.num_competitive.divide(df.highway_length) * 
                              geography_utils.FEET_PER_MI),
    )
    
    # Add in highways geometry
    highways = catalog.highways_cleaned.read()

    gdf = pd.merge(
        highways.assign(geometry=highways.geometry.buffer(200)).drop(
            columns = ["NB", "SB", "EB", "WB"]),
        df,
        on = ["Route", "County", "District"],
        how = "inner",
        validate = "m:1"
    ).rename(columns = {
        "highway_length_x": "highway_length_routetype",
        "highway_length_y": "highway_length_route",
    })
    
    return gdf

In [ ]:
operator_stats, hwy_stats = setup_corridors_stats.aggregated_transit_hwy_stats()
gdf = process_hwy_stats(hwy_stats)

In [ ]:
def labeling(word):
    rename_dict = {
        "pct_parallel": "% parallel routes",
        "pct_competitive": "% competitive routes"
    }
    
    if word in rename_dict.keys():
        word = rename_dict[word]
    else:
        word = word.replace('_', ' ').title()
        
    return word

def make_scatterplot(df, x_col, y_col):
    
    if "pct" in x_col:
        x_p50 = 0.5
        y_p50 = 0.5
    else:
        x_p50 = 0.1
        y_p50 = 0.1
    
    chart = (alt.Chart(df)
             .mark_circle(size=30, opacity=0.6)
             .encode(
                 x=alt.X(f"{x_col}:Q", title=labeling(x_col)),
                 y=alt.Y(f"{y_col}:Q", title=labeling(y_col)),
                 color=alt.Color("District:N"),
                 tooltip=["Route", "County", "District",
                          "num_parallel", "pct_parallel", "parallel_per_mi",
                          "num_competitive", "pct_competitive", "competitive_per_mi",
                 ]
             ).interactive()
             .properties(title = f"{labeling(x_col)} vs {labeling(y_col)}")
    )
    
    horiz_line = (alt.Chart(df.assign(y_p50=y_p50))
                  .mark_rule(strokeDash=[2,3])
                  .encode(
                      y="y_p50:Q",
                      color=alt.value("black")
                  )
    )
    
    vertical_line = (alt.Chart(df.assign(x_p50=x_p50))
                     .mark_rule(strokeDash=[2,3])
                     .encode(
                         x="x_p50:Q",
                         color=alt.value("black")
                     )
    )
    
    combined = chart + horiz_line + vertical_line
    return combined

In [ ]:
make_scatterplot(gdf[gdf.pct_parallel > 0], "pct_parallel", "pct_competitive")

In [ ]:
make_scatterplot(gdf[gdf.pct_parallel > 0], "parallel_per_mi", "competitive_per_mi")

## Highhways with Zero Parallel Routes

These indicate a dearth of bus transit service.

Sort in descending order where there are more bus routes, just that these aren't parallel bus routes, and then sort in decreasing length of highway corridor.

Show top 20 and make district maps.

In [ ]:
top20_zero_parallel = (gdf[gdf.pct_parallel==0]
                       .sort_values(["count_route_id", "highway_length_routetype"], 
                                    ascending = [False, False]).head(20)
                      )

In [ ]:
FIG_HEIGHT = 900
FIG_WIDTH = 700

PLOT_COL = "count_route_id"
POPUP_DICT = {
    "Route": "Hwy Route",
    "County": "County",
    "District": "District",
    "RouteType": "Route Type",
    "count_route_id": "# transit routes",
}

In [ ]:
def make_district_map(gdf, district, chart_title):
    display(Markdown(f"### District {district}"))
    
    # Display table first
    subset = gdf[gdf.District==district].to_crs(geography_utils.WGS84)
    
    display_cols = [
        "Route", "County", "District", "RouteType", 
        "count_route_id"
    ]
    display(subset.sort_values(["count_route_id", "highway_length_routetype"], 
                ascending=[False, False]).reset_index()[
        display_cols].rename(columns = POPUP_DICT))
    
    district_centroid = (subset
                    .geometry.centroid
                    .iloc[0]
                   )
    
    COLORSCALE = branca.colormap.step.Accent_08.scale(
        vmin = subset.count_route_id.min(),
        vmax=subset.count_route_id.max(),
    )
    
    m = map_utils.make_folium_choropleth_map(
        subset,
        plot_col = PLOT_COL,
        popup_dict = POPUP_DICT,
        tooltip_dict = POPUP_DICT,
        colorscale = COLORSCALE,
        fig_width = FIG_WIDTH,
        fig_height = FIG_HEIGHT,
        zoom = 10,
        centroid = [round(district_centroid.y,2), 
                    round(district_centroid.x, 2)],
        title=chart_title,
        legend_name="Legend",
    )

    return m


In [ ]:
district_list = sorted(list(top20_zero_parallel.District.unique()))
for i in district_list:
    chart_title = f"District {i}: Highway Corridors with No Parallel Transit Routes"
    m = make_district_map(top20_zero_parallel, i, chart_title)
    display(m)

## Show low competitive routes

This is another case for Caltrans Express Bus, maybe not necessarily from Caltrans, but a case for operators who have parallel routes to perhaps make some of those more competitive with car

Also include high parallel routes? Bottom right of quandrant or just bottom of chart

In [ ]:
with_parallel = gdf[gdf.pct_parallel > 0]

#parallel_p75 = with_parallel.pct_parallel.quantile(0.75)
competitive_p25 = with_parallel.pct_competitive.quantile(0.25)

In [ ]:
with_parallel2 = with_parallel[
    #(with_parallel.pct_parallel > parallel_p75) & 
    (with_parallel.pct_competitive < competitive_p25)
]

In [ ]:
district_list = sorted(list(with_parallel2.District.unique()))
for i in district_list: 
    chart_title = f"District {i}: Highway Corridors with Few Competitive Transit Routes"
    m = make_district_map(with_parallel2, i, chart_title)
    display(m)